In [25]:
import os
import argparse
import tools.test as test
import warnings

from mmcv import Config, DictAction
from mmdet.datasets import replace_ImageToTensor, build_dataset


In [9]:
def parse_args():
    parser = argparse.ArgumentParser(
        description="MMDet test (and eval) a model"
    )
    parser.add_argument("config", help="test config file path")
    parser.add_argument("checkpoint", help="checkpoint file")
    parser.add_argument("--out", help="output result file in pickle format")
    parser.add_argument(
        "--fuse-conv-bn",
        action="store_true",
        help="Whether to fuse conv and bn, this will slightly increase"
        "the inference speed",
    )
    parser.add_argument(
        "--format-only",
        action="store_true",
        help="Format the output results without perform evaluation. It is"
        "useful when you want to format the result to a specific format and "
        "submit it to the test server",
    )
    parser.add_argument(
        "--eval",
        type=str,
        nargs="+",
        help='evaluation metrics, which depends on the dataset, e.g., "bbox",'
        ' "segm", "proposal" for COCO, and "mAP", "recall" for PASCAL VOC',
    )
    parser.add_argument("--show", action="store_true", help="show results")
    parser.add_argument(
        "--show-dir", help="directory where results will be saved"
    )
    parser.add_argument(
        "--gpu-collect",
        action="store_true",
        help="whether to use gpu to collect results.",
    )
    parser.add_argument(
        "--tmpdir",
        help="tmp directory used for collecting results from multiple "
        "workers, available when gpu-collect is not specified",
    )
    parser.add_argument("--seed", type=int, default=0, help="random seed")
    parser.add_argument(
        "--deterministic",
        action="store_true",
        help="whether to set deterministic options for CUDNN backend.",
    )
    parser.add_argument(
        "--cfg-options",
        nargs="+",
        action=DictAction,
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file. If the value to "
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        "Note that the quotation marks are necessary and that no white space "
        "is allowed.",
    )
    parser.add_argument(
        "--options",
        nargs="+",
        action=DictAction,
        help="custom options for evaluation, the key-value pair in xxx=yyy "
        "format will be kwargs for dataset.evaluate() function (deprecate), "
        "change to --eval-options instead.",
    )
    parser.add_argument(
        "--eval-options",
        nargs="+",
        action=DictAction,
        help="custom options for evaluation, the key-value pair in xxx=yyy "
        "format will be kwargs for dataset.evaluate() function",
    )
    parser.add_argument(
        "--launcher",
        choices=["none", "pytorch", "slurm", "mpi"],
        default="none",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument("--result_file", type=str, default=None)
    parser.add_argument("--show_only", action="store_true")
    args = parser.parse_args()
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    if args.options and args.eval_options:
        raise ValueError(
            "--options and --eval-options cannot be both specified, "
            "--options is deprecated in favor of --eval-options"
        )
    if args.options:
        warnings.warn("--options is deprecated in favor of --eval-options")
        args.eval_options = args.options
    return args


In [16]:
import sys
# 기본 설정 파일과 체크포인트 경로 설정
sys.argv = ['test_data', 
            'projects/configs/BridgeAD_small_stage2.py',
            'ckpt/resnet50-19c8e357.pth']
args = parse_args()
cfg = Config.fromfile(args.config)

In [26]:
dataset = build_dataset(cfg.data.test)

{'version': 'v1.0-mini'}


In [30]:
args.checkpoint

'ckpt/resnet50-19c8e357.pth'

In [28]:
cfg.model

{'type': 'SparseDrive',
 'use_grid_mask': True,
 'use_deformable_func': True,
 'img_backbone': {'type': 'ResNet',
  'depth': 50,
  'num_stages': 4,
  'frozen_stages': -1,
  'norm_eval': False,
  'style': 'pytorch',
  'with_cp': True,
  'out_indices': (0, 1, 2, 3),
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'pretrained': 'ckpt/resnet50-19c8e357.pth'},
 'img_neck': {'type': 'FPN',
  'num_outs': 4,
  'start_level': 0,
  'out_channels': 256,
  'add_extra_convs': 'on_output',
  'relu_before_extra_convs': True,
  'in_channels': [256, 512, 1024, 2048]},
 'head': {'type': 'SparseDriveHead',
  'task_config': {'with_det': True,
   'with_map': True,
   'with_motion_plan': True},
  'use_motion_for_det': True,
  'det_head': {'type': 'Sparse4DHead',
   'cls_threshold_to_reg': 0.05,
   'decouple_attn': True,
   'instance_bank': {'type': 'InstanceBank',
    'num_anchor': 900,
    'embed_dims': 256,
    'anchor': 'data/kmeans/kmeans_det_900.npy',
    'anchor_handler': {'type': 'SparseBox3DK

In [29]:
args.checkpoint

'ckpt/resnet50-19c8e357.pth'